Classify input images

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image

input_shape = (224, 224, 3)

def load_and_preprocess_image(image_path):
    """Loads a TIFF image, resizes it, and normalizes pixel values."""
    # Load the image using Pillow (PIL)
    img = Image.open(image_path)

    # Resize to the model's input shape
    img = img.resize(input_shape[:2])

    # Convert image to a NumPy array and normalize
    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0  

    # Expand dimensions for a batch of size 1 
    img_array = np.expand_dims(img_array, axis=0) 
    
    return img_array

def predict_image(model, image_path):
  """Loads, preprocesses, and gets prediction for input image"""
  img_array = load_and_preprocess_image(image_path)

  # Get predictions from the model
  predictions = model.predict(img_array)

  # Since 'sigmoid' activation gives probability, the class is 1 if probability > 0.5
  predicted_class = int(predictions[0][0] <= 0.5)  

  # You can add labels here if needed
  label = "natural" if predicted_class == 0 else "built"

  return label
# Example usage:
#correct=0
#for i in range(1,111):
#    image_path = rf"C:\Users\ADMIN\Desktop\Coastline Detection\Coastline-Detection-Summer-Project-master\ML Based\split-built\train\images\{i}.tiff"
#    predicted_label = predict_image(model, image_path)
#    if predicted_label == "built":
#       correct+=1
#    #print("Predicted label:", predicted_label)
#print(correct) 


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from skimage.transform import resize
from keras.models import load_model
import os
# image properties
IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 3

model_built = load_model('UNet-built.h5')
model_natural = load_model('UNet-natural.h5')
classification_model = model = keras.models.load_model('natural-built-classifier.h5')
current_dir = rf'C:\Users\ADMIN\Desktop\Coastline Detection\Coastline-Detection-Summer-Project-master\ML Based\test\completeFlow'
image_dir = rf'C:\Users\ADMIN\Desktop\Coastline Detection\Coastline-Detection-Summer-Project-master\ML Based\test\completeFlow\images'

# Process each TIFF image in the directory
for filename in os.listdir(image_dir):
    if filename.endswith('.tiff'):
        image_path = os.path.join(image_dir, filename)
        model = model_built
        if predict_image(classification_model, image_path)== "natural":
            model = model_natural
        # Read the original image
        img = imread(image_path)[:,:,:IMG_CHANNELS]
        original_size = img.shape[:2]  # Store original size

        # Resize the image for the model
        img_resized = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

        # Place into an array for prediction (batch size of 1)
        X_test = np.expand_dims(img_resized, axis=0) 

        preds_test = model.predict(X_test, verbose=1)
        preds_test_t = (preds_test > 0.5).astype(np.uint8)

        # Resize the predicted image back to original size
        pred_image_resized = resize(np.squeeze(preds_test_t[0]), original_size, mode='constant', preserve_range=True)
        pred_image_resized = (pred_image_resized > 0.5).astype(np.uint8) * 255

        # Get image number from the filename
        image_num = int(filename.split('.')[0])  

        # Save the predicted image
        save_path = rf'C:\Users\ADMIN\Desktop\Coastline Detection\Coastline-Detection-Summer-Project-master\ML Based\test\completeFlow\results\{image_num}_pred.tiff'
        imsave(save_path, pred_image_resized.astype(np.uint8))


1/1 [==============================] - 0s 7ms/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15848\3037536542.py:47: UserWarning: C:\Users\ADMIN\Desktop\Coastline Detection\Coastline-Detection-Summer-Project-master\ML Based\test\completeFlow\results\142_pred.tiff is a low contrast image
  imsave(save_path, pred_image_resized.astype(np.uint8))


1/1 [==============================] - 0s 20ms/step


In [3]:
from PIL import Image
import numpy as np
import os

def calculate_iou(image_path, mask_path):
    # Load image and mask using PIL
    image = Image.open(image_path)
    mask = Image.open(mask_path)

    # Convert images to numpy arrays
    image_array = np.array(image)
    mask_array = np.array(mask)

    # Handle multi-channel images or masks
    if image_array.ndim == 3:
        image_array = image_array[:, :, 0]  # Use the first channel of the image
    if mask_array.ndim == 3:
        mask_array = mask_array[:, :, 0]    # Use the first channel of the mask

    # Binarize images (assuming 255 is foreground)
    image_array = np.where(image_array == 255, 1, 0)
    mask_array = np.where(mask_array == 255, 1, 0)

    # Calculate intersection and union
    intersection = np.logical_and(image_array, mask_array)
    union = np.logical_or(image_array, mask_array)

    # Calculate IoU (handle potential division by zero)
    iou = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0

    return iou

ious = []  # Store IoU scores
image_dir=rf'C:\Users\ADMIN\Desktop\Coastline Detection\Coastline-Detection-Summer-Project-master\ML Based\test\completeFlow\images'
# Process each TIFF image in the directory
for filename in os.listdir(image_dir):
    if filename.endswith('.tiff'):
        image_num = int(filename.split('.')[0])  # Get image number
        image_path = rf'C:\Users\ADMIN\Desktop\Coastline Detection\Coastline-Detection-Summer-Project-master\ML Based\test\completeFlow\masks\{image_num}_mask.tiff'
        mask_path = rf'C:\Users\ADMIN\Desktop\Coastline Detection\Coastline-Detection-Summer-Project-master\ML Based\test\completeFlow\results\{image_num}_pred.tiff'

        iou = calculate_iou(image_path, mask_path)
        ious.append(iou)

        
        print(f"Image {image_num}: IoU = {iou}")

# Calculate average IoU
if len(ious) > 0:
    average_iou = sum(ious) / len(ious)
    print(f"Average IoU = {average_iou}")
else:
    print("No images found or IoU couldn't be calculated.") 


Image 1: IoU = 0.9303935630299782
Image 141: IoU = 0.7571454224952083
Image 142: IoU = 0.907267398210208
Image 143: IoU = 0.9143199172987111
Image 144: IoU = 0.9358271048754383
Image 145: IoU = 0.8222004928744624
Image 2: IoU = 0.9616900163614459
Image 223: IoU = 0.9799861503706478
Image 224: IoU = 0.9818011755332048
Image 225: IoU = 0.6536077137125478
Image 226: IoU = 0.6897221706577563
Image 227: IoU = 0.6635245901639344
Image 3: IoU = 0.8641342409726308
Image 4: IoU = 0.9839699999430955
Image 5: IoU = 0.9391719236604648
Average IoU = 0.865650792010649
